In [24]:
# In[ ]:
# ** import package **
import os
import sys
import json
import pathlib
sys.path.append("..")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import traceback
from tqdm import tqdm
from datetime import timedelta
from _utils.customlogger import customlogger as CL

pd.set_option('display.max_colwidth', -1)  #각 컬럼 width 최대로 
pd.set_option('display.max_rows', 50)      # display 50개 까지 

/home/suncheol/code/PPFL/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Passing a negative integer is deprecated in version 1.0 and will not be supported in future version. Instead, use None to not limit the column width.
  app.launch_new_instance()


In [25]:
# In[ ]:
# ** loading path info **
current_dir = pathlib.Path.cwd()
parent_dir = current_dir.parent
curr_file_name = os.path.splitext(os.path.basename(os.path.abspath('')))[0]
data_dir = pathlib.Path('{}/data/'.format(parent_dir))
pathlib.Path.mkdir(data_dir, mode=0o777, parents=True, exist_ok=True)

In [26]:
# ** loading data **
# MICU_df = pd.read_feather('{}/MICU_df.feather'.format(data_dir))
# SICU_df = pd.read_feather('{}/SICU_df.feather'.format(data_dir))
# NSICU_df = pd.read_feather('{}/NSICU_df.feather'.format(data_dir))
icu_list = ['MSICU', 'CTICU', 'SICU', 'CCUCTICU', 'MICU', 'NICU', 'CICU', 'CSICU']
icu_df_dict = {}
for icu in icu_list:
    icu_df_dict[icu] = pd.read_feather('{}/{}_df.feather'.format(data_dir, icu))

In [27]:
# ** calc Glasgow Coma Scale **
def calculate_gcs(eyes, motor, verbal, meds):
    if meds == 1:
        return np.nan
    return eyes + motor + verbal

for icu in icu_list:
    icu_df_dict[icu].loc[:,'gcs'] = icu_df_dict[icu].apply(lambda x: calculate_gcs(x['eyes'], x['motor'], x['verbal'], x['meds']), axis=1)
    icu_df_dict[icu].drop(['eyes', 'motor', 'verbal', 'meds'], axis=1, inplace=True)

In [28]:
all_cols_df = pd.read_feather('{}/all_cols_df.feather'.format(data_dir))

In [29]:
def fill_missing_values(df, cols_df):
    
    pat_feature_list = list(set(df.columns) & set(cols_df.loc[cols_df['concept_table']=='pat'].concept_name))
    apc_feature_list = list(set(df.columns) & set(cols_df.loc[cols_df['concept_table']=='apc'].concept_name))
    med_feature_list = list(set(df.columns) & set(cols_df.loc[cols_df['concept_table']=='med'].concept_name))
    lab_feature_list = list(set(df.columns) & set(cols_df.loc[cols_df['concept_table']=='lab'].concept_name))
    adm_feature_list = list(set(df.columns) & set(cols_df.loc[cols_df['concept_table']=='adm'].concept_name))

    print(len(pat_feature_list), len(apc_feature_list), len(med_feature_list), len(lab_feature_list), len(adm_feature_list))
    print('total : ', len(pat_feature_list)+len(apc_feature_list)+len(med_feature_list)+len(lab_feature_list)+len(adm_feature_list))

    ## patient
    df = df[~df['hospitaldischargestatus'].isna()]
    expired_loc = df['hospitaldischargestatus']=='Expired'
    alive_loc = df['hospitaldischargestatus']=='Alive'
    df['ethnicity'] = df['ethnicity'].fillna('Other/Unknown')
    df['apacheadmissiondx'] = df['apacheadmissiondx'].fillna('None')
    df['unitdischargestatus'] = df['unitdischargestatus'].fillna('None')
    df['hospitaladmitsource'] = df['hospitaladmitsource'].fillna('None')
    df['hospitaldischargelocation'] = df['hospitaldischargelocation'].fillna('None')
    df['unitadmitsource'] = df['unitadmitsource'].fillna('None')
    df['unitdischargelocation'] = df['unitdischargelocation'].fillna('None')
    other_cols = ['admissionheight', 'admissionweight', 'dischargeweight', 'hospitaladmitoffset', 'gcs']
    df.loc[expired_loc,other_cols] = df.loc[expired_loc,other_cols].fillna(df.loc[expired_loc,other_cols].mean())
    df.loc[alive_loc,other_cols] = df.loc[alive_loc,other_cols].fillna(df.loc[alive_loc,other_cols].mean())
    
    ## apacheapsvar
    df.loc[:,apc_feature_list] = df.loc[:,apc_feature_list].replace(-1, np.nan)
    df.loc[expired_loc,apc_feature_list] = df.loc[expired_loc,apc_feature_list].fillna(df.loc[expired_loc,apc_feature_list].mean())
    df.loc[alive_loc,apc_feature_list] = df.loc[alive_loc,apc_feature_list].fillna(df.loc[alive_loc,apc_feature_list].mean())
    
    ## lab, medication, admissiondrug
    df.loc[expired_loc,lab_feature_list] = df.loc[expired_loc,lab_feature_list].fillna(df.loc[expired_loc,lab_feature_list].mean())
    df.loc[alive_loc,lab_feature_list] = df.loc[alive_loc,lab_feature_list].fillna(df.loc[alive_loc,lab_feature_list].mean())
    df[med_feature_list] = df[med_feature_list].fillna(0)
    df[adm_feature_list] = df[adm_feature_list].fillna(0)
    
    return df

for icu in icu_list:
    icu_df_dict[icu] = fill_missing_values(icu_df_dict[icu], all_cols_df)

29 22 18 28 1
total :  98


/home/suncheol/code/PPFL/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/home/suncheol/code/PPFL/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  app.launch_new_instance()
/home/suncheol/code/PPFL/.venv/lib/python3.7/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = valu

29 22 140 24 1
total :  216
29 22 63 20 2
total :  136
29 22 46 23 1
total :  121
29 22 21 28 0
total :  100
29 22 70 18 0
total :  139
29 22 41 26 2
total :  120
29 22 95 21 0
total :  167


In [30]:
icu_df_dict['SICU']

,patientunitstayid,patienthealthsystemstayid,gender,age,ethnicity,hospitalid,wardid,apacheadmissiondx,admissionheight,hospitaladmittime24,...,calcium,chloride,fentaNYL,fentaNYL citrate (PF),hydrALAZINE,magnesium,morphine,platelets x 1000,potassium,gcs
0,141296,129026,Male,63,Caucasian,73,85,"Hypovolemia (including dehydration, Do not include shock states)",162.6,04:18:00,...,8.416667,95.500000,0.0,0.0,0.0,2.037001,0.0,437.833333,5.050000,15.0
1,141297,129026,Male,63,Caucasian,73,85,"Sepsis, pulmonary",162.6,04:18:00,...,9.966667,97.333333,0.0,0.0,0.0,3.200000,0.0,440.000000,7.066667,15.0
2,141314,129039,Male,45,Caucasian,73,85,"Aneurysm, abdominal aortic; with rupture",170.2,00:41:00,...,10.300000,109.000000,0.0,0.0,0.0,2.350000,0.0,172.500000,6.533333,3.0
3,141675,129327,Female,58,Caucasian,68,103,"Infarction, acute myocardial (MI)",151.8,13:43:00,...,8.500000,108.000000,0.0,0.0,0.0,1.949268,0.0,232.000000,4.200000,15.0
4,141708,129348,Male,82,Caucasian,68,103,Head/extremity trauma,188.0,14:00:00,...,8.300000,105.500000,0.0,0.0,0.0,1.949268,0.0,188.000000,3.800000,15.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10699,3353007,2742882,Female,74,Caucasian,458,1111,Chest/extremity trauma,157.5,09:01:00,...,8.700000,97.333333,0.0,0.0,0.0,1.625000,0.0,224.333333,4.366667,15.0
10700,3353015,2742889,Male,19,African American,458,1111,Abdomen/pelvis trauma,165.1,14:24:00,...,7.464516,101.258065,0.0,0.0,0.0,1.838095,0.0,385.358974,4.470968,15.0
10701,3353031,2742903,Male,45,Caucasian,458,1111,Chest/thorax only trauma,188.0,22:23:00,...,8.075000,109.250000,0.0,0.0,0.0,1.766667,0.0,226.750000,5.283333,11.0
10702,3353043,2742913,Female,27,African American,458,1111,Cesarean section,165.1,21:18:00,...,9.000000,103.000000,0.0,0.0,0.0,1.949268,0.0,257.666667,4.400000,15.0


In [31]:
def resumetable(df):
    print(f'data frame shape: {df.shape}')
    summary = pd.DataFrame(df.dtypes, columns=['data_type'])
    summary = summary.reset_index()
    summary = summary.rename(columns={'index': 'feature'})
    summary['n_missingvalues'] = df.isnull().sum().values
    summary['n_missingrate'] = df.isnull().sum().values/len(df)
    summary['n_eigenvalues'] = df.nunique().values
    print(len(summary[summary['n_missingvalues'] != 0].feature))
    print(list(summary[summary['n_missingvalues'] != 0].feature))
    return summary

icu_summary_dict = {}
for icu in icu_list:
    icu_summary_dict[icu] = resumetable(icu_df_dict[icu])

data frame shape: (92369, 94)
2
['gender', 'age']
data frame shape: (5578, 212)
0
[]
data frame shape: (10649, 132)
0
[]
data frame shape: (14201, 117)
0
[]
data frame shape: (14550, 96)
0
[]
data frame shape: (12710, 135)
0
[]
data frame shape: (10594, 116)
0
[]
data frame shape: (6320, 163)
1
['age']


In [32]:
icu_summary_dict['SICU']

,feature,data_type,n_missingvalues,n_missingrate,n_eigenvalues
0,patientunitstayid,int64,0,0.0,10649
1,patienthealthsystemstayid,int64,0,0.0,10155
2,gender,object,0,0.0,4
3,age,object,0,0.0,77
4,ethnicity,object,0,0.0,6
...,...,...,...,...,...
127,magnesium,float64,0,0.0,1390
128,morphine,float64,0,0.0,2
129,platelets x 1000,float64,0,0.0,5479
130,potassium,float64,0,0.0,2368


In [33]:
# ** check duplicated columns **
def check_duplicated_columns(df):
    cols = list(df.columns)
    dup = [x for i, x in enumerate(cols) if i != cols.index(x)]
    if len(dup) > 0 :
        print(dup)

# ** make to dict rename columns **
def get_rename_dict():
    eicu_mapping = pd.read_csv('eicu_mapping.csv')
    eicu_mapping.replace(['\r\n', '\n'], '', inplace=True)
    eicu_mapping_diff = eicu_mapping[eicu_mapping['concept_name'] != eicu_mapping['feature_new_name']]
    rename_dict = dict(zip(eicu_mapping_diff.concept_name, eicu_mapping_diff.feature_new_name))
    print(rename_dict)
    return rename_dict

for icu in icu_list:
    rename_dict = get_rename_dict()
    icu_df_dict[icu].rename(rename_dict, axis='columns', inplace=True)
    icu_df_dict[icu].reset_index(drop=True, inplace=True)
    check_duplicated_columns(icu_df_dict[icu])
    data_ps_dir = pathlib.Path.joinpath(data_dir, 'ps')
    pathlib.Path.mkdir(data_ps_dir, mode=0o777, parents=True, exist_ok=True)
    icu_df_dict[icu].to_feather('{}/{}_df.feather'.format(data_ps_dir, icu))

{'BUN': 'bun', 'CREATININE                                                                                                                                                                                                                                                     ': 'creatinine', 'GLUCOSE                                                                                                                                                                                                                                                        ': 'glucose', 'HCO3': 'hco3-', 'Hct': 'hct', 'POTASSIUM                                                                                                                                                                                                                                                      ': 'potassium', 'RBC': 'rbc count', 'Temperature': 'temperature', 'pH': 'ph', 'urinary creatinine': 'creatinine'}
{'BUN': 'bun', 'CREATININE               

In [22]:
for icu in icu_list:
    data_ps_dir = pathlib.Path.joinpath(data_dir, 'ps')
    pathlib.Path.mkdir(data_ps_dir, mode=0o777, parents=True, exist_ok=True)
    icu_df_dict[icu] = pd.read_feather('{}/{}_df.feather'.format(data_ps_dir, icu))
    print('icu : ', icu, ' len(columns) : ', len(icu_df_dict[icu].columns), ' len(df) : ', len(icu_df_dict[icu]))

icu :  MSICU  len(columns) :  97  len(df) :  93339
icu :  CTICU  len(columns) :  215  len(df) :  5618
icu :  SICU  len(columns) :  135  len(df) :  10704
icu :  CCUCTICU  len(columns) :  120  len(df) :  14251
icu :  MICU  len(columns) :  99  len(df) :  14651
icu :  NICU  len(columns) :  138  len(df) :  12857
icu :  CICU  len(columns) :  119  len(df) :  10704
icu :  CSICU  len(columns) :  166  len(df) :  6364
